# 0. Configuration

In [1]:
# links to shared data MovieLens
# source on kaggle: https://www.kaggle.com/code/quangnhatbui/movie-recommender/data
RATINGS_SMALL_URL = 'https://drive.google.com/file/d/1BlZfCLLs5A13tbNSJZ1GPkHLWQOnPlE4/view?usp=share_link'
MOVIES_METADATA_URL = 'https://drive.google.com/file/d/19g6-apYbZb5D-wRj4L7aYKhxS-fDM4Fb/view?usp=share_link'

# 1. Modules and functions

In [2]:
# just to make it available to download w/o SSL verification
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import random
import numpy as np
import pandas as pd
import datetime as dt
from itertools import permutations

import torch
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')


RANDOM_STATE = 42

## 1.1. Helper functions to avoid copy paste

In [3]:
def read_csv_from_gdrive(url):
    """
    gets csv data from a given url (taken from file -> share -> copy link)
    :url: example https://drive.google.com/file/d/1BlZfCLLs5A13tbNSJZ1GPkHLWQOnPlE4/view?usp=share_link
    """
    file_id = url.split('/')[-2]
    file_path = 'https://drive.google.com/uc?export=download&id=' + file_id
    data = pd.read_csv(file_path)

    return data

# 2. Main

## 2.1. RankNet

In [4]:
class RankNet(torch.nn.Module):
    def __init__(self, input_features_len, hidden_dim = 10):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_features_len, self.hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(self.hidden_dim, 1),
        )
        
        self.out_activation = torch.nn.Sigmoid()

    def forward(self, input_1, input_2):
        logits_1 = self.predict(input_1)
        logits_2 = self.predict(input_2)
        
        logits_diff = logits_1 - logits_2
        out = self.out_activation(logits_diff)

        return out
    
    def predict(self, inp):
        logits = self.model(inp)
        return logits

In [5]:
model = RankNet(input_features_len = 8)
model

RankNet(
  (model): Sequential(
    (0): Linear(in_features=8, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=1, bias=True)
  )
  (out_activation): Sigmoid()
)

In [6]:
input_1, input_2 = torch.rand(4, 8), torch.rand(4, 8)
input_2

tensor([[0.7056, 0.4304, 0.7819, 0.9265, 0.5223, 0.5009, 0.1768, 0.3562],
        [0.9407, 0.6271, 0.3754, 0.8852, 0.2320, 0.5111, 0.8795, 0.7290],
        [0.0043, 0.4087, 0.9889, 0.2314, 0.0170, 0.7701, 0.5280, 0.8738],
        [0.4480, 0.0341, 0.1732, 0.2917, 0.0119, 0.4544, 0.3268, 0.5849]])

In [7]:
preds = torch.sort(model(input_1, input_2), descending = True, dim = 0)
preds[0]

tensor([[0.5334],
        [0.4944],
        [0.4824],
        [0.4794]], grad_fn=<SortBackward0>)

## 2.2. ListNet

In [8]:
movies_to_rank = {'The Godfather', 'Avatar', 'Ozark'}
permutations_list = list(permutations(movies_to_rank))

for i in permutations_list:
    print(i)

('Ozark', 'Avatar', 'The Godfather')
('Ozark', 'The Godfather', 'Avatar')
('Avatar', 'Ozark', 'The Godfather')
('Avatar', 'The Godfather', 'Ozark')
('The Godfather', 'Ozark', 'Avatar')
('The Godfather', 'Avatar', 'Ozark')


In [23]:
pi = random.choice(permutations_list)
print(pi)

('The Godfather', 'Avatar', 'Ozark')


In [25]:
np.random.seed(RANDOM_STATE)
scores_dict = {x: np.random.randn(1)[0] for x in movies_to_rank}  
print(scores_dict)

# unpack pi and assign movies to scores
score_movie_pos_1, score_movie_pos_2, score_movie_pos_3 = scores_dict[pi[0]], scores_dict[pi[1]], scores_dict[pi[2]]


{'Ozark': 0.4967141530112327, 'Avatar': -0.13826430117118466, 'The Godfather': 0.6476885381006925}


In [26]:
first_term = np.exp(score_movie_pos_1) / (np.exp(score_movie_pos_1) + np.exp(score_movie_pos_2)\
                                         + np.exp(score_movie_pos_3))

second_term = np.exp(score_movie_pos_2) / (np.exp(score_movie_pos_2) + np.exp(score_movie_pos_3))

third_term = np.exp(score_movie_pos_3) / np.exp(score_movie_pos_3)

print(f'First term is: {first_term}')
print(f'Second term is: {second_term}')
print(f'Third term is: {third_term}')

First term is: 0.4318619033836114
Second term is: 0.3463825470936087
Third term is: 1.0


$P_{s}(<The Godfather, Avatar, Ozark>) = \prod^3_{j = 1} \frac {\phi(s_{\pi(j)})} {\sum^3_{k = j} \phi(s_{\pi(k)})}$ which is equal to

In [28]:
permutation_proba = first_term * second_term * third_term

print(f'Permutation probability is: {permutation_proba}')


Permutation probability is: 0.14958942608670928


## 2.3. CatBoost Ranker

### 2.3.1. Load Data

`interactions` dataset shows list of movies that users watched, along with given ratings:

In [ ]:
# interactions data
interactions = read_csv_from_gdrive(RATINGS_SMALL_URL)
interactions.head()

`movies_metadata` dataset shows the list of movies existing on OKKO platform:

In [ ]:
# information about films etc
movies_metadata = read_csv_from_gdrive(MOVIES_METADATA_URL)
movies_metadata.head(3)

In [ ]:
movies_metadata['id'] = movies_metadata['id'].astype(str)
interactions['movieId'] = interactions['movieId'].astype(str)

In [ ]:
movies_metadata.rename(columns = {'id': 'movieId'}, inplace = True)

In [ ]:
# leave only those films that intersect with each other
interactions_filtered = interactions.loc[interactions['movieId'].isin(movies_metadata['id'])]
print(interactions.shape, interactions_filtered.shape)

### 2.3.2 Data Preparation

In [ ]:
TEST_SIZE = .25

In [ ]:
# convert timestamp to date
interactions_filtered['dttm'] = interactions_filtered['timestamp']\
                                .apply(lambda x: pd.to_datetime(dt.datetime.fromtimestamp(x).strftime('%Y-%m-%d')))

Let's calculate some basic features, but keep in mind that our data of historical ratings depends on time.
We need to avoid data leak -- use future values in past data

In [ ]:
ITEM_FEATURES_LIST = ['revenue', 'budget', 'runtime']

In [ ]:
# calculate avg ratings by users and items daily
daily_users_feature = interactions_filtered.groupby(['userId', 'dttm']).agg({'rating': 'mean',
                                              'movieId': 'count'})\
                                  .reset_index().sort_values(['userId', 'dttm'])\
                                  .rename(columns = {'rating': 'user_mean_rating',
                                                     'movieId': 'user_watch_count'})


daily_users_feature['dttm'] = daily_users_feature['dttm'].apply(lambda x: x + dt.timedelta(days = 1))
daily_users_feature.loc[daily_users_feature['userId'] == 3]

In [ ]:
daily_users_feature.loc[daily_users_feature['userId'] == 671]

In [ ]:
cumulative_total_cnt = daily_users_feature.set_index('dttm').groupby(['userId'])['user_watch_count']\
                        .rolling(window = 3, min_periods = 1).sum()\
                        .reset_index()[['userId', 'dttm', 'user_watch_count']]\
                        .rename(columns = {'user_watch_count': 'user_total_watch_count_last_3_days'})

In [ ]:
# merge item features
main_df = pd.merge(
    interactions_filtered, movies_metadata[['movieId']+ ITEM_FEATURES_LIST],
    how = 'left', on = 'movieId'
                   ).drop_duplicates().reset_index(drop = True)
assert main_df.shape[0] == interactions_filtered.shape[0]

In [ ]:
main_df = main_df.sort_values('dttm').reset_index(drop = True)
daily_users_feature = daily_users_feature.sort_values('dttm').reset_index(drop = True)
cumulative_total_cnt = cumulative_total_cnt.sort_values('dttm').reset_index(drop = True)

In [ ]:
# merge user features with watch count
main_df = pd.merge_asof(
    main_df, daily_users_feature,
    on = 'dttm', by = 'userId',
    direction = 'backward',
    allow_exact_matches = True
    )
assert main_df.shape[0] == interactions_filtered.shape[0]

In [ ]:
main_df = pd.merge_asof(
    main_df, cumulative_total_cnt,
    on = 'dttm', by = 'userId',
    direction = 'backward',
    allow_exact_matches = True
    )
assert main_df.shape[0] == interactions_filtered.shape[0]

In [ ]:
cumulative_total_cnt.loc[cumulative_total_cnt['userId'] == 671]

In [ ]:
# tmp  = main_df.loc[main_df['userId'] == 671][['userId', 'dttm']]
# pd.merge_asof(
#     tmp.sort_values('dttm'), cumulative_total_cnt.sort_values('dttm'),
#     on = 'dttm', by = 'userId', direction = 'backward',
#     allow_exact_matches = True).sort_values('dttm')

In [ ]:
# anyway we left some NaN
main_df.isnull().sum() / len(main_df) 

In [ ]:
FINAL_FEATURES_LIST = ['revenue', 'budget', 'runtime', 'user_mean_rating',
                       'user_watch_count', 'user_total_watch_count_last_3_days']

In [ ]:
ID_COLS = ['userId', 'movieId']

In [ ]:
TARGET = 'rating'

In [ ]:
X = main_df[ID_COLS + FINAL_FEATURES_LIST]
y = main_df[TARGET]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size = TEST_SIZE,
    random_state = RANDOM_STATE)

print(f'Shape of train set X, y: {X_train.shape}, {len(y_train)}')
print(f'Shape of train set X, y: {X_test.shape}, {len(y_test)}')

### 2.3.3. Train Model

In [ ]:
# init model
model = CatBoostRegressor(
    loss_function = 'MAE',
    iterations = 2000,
    learning_rate = .1,
    depth = 6,
    verbose = False
)

In [ ]:
model.fit(
    X_train, y_train,
    eval_set=(X_test, y_test),
    early_stopping_rounds = 20 # to avoid overfitting,
)

In [ ]:
model.best_score_